# Homework 4
In this homework assignment, your friends in forecast department are in trouble and you will help them implement a number of performance metrics used in evaluating their forecasts. 

There are many types of forecasts, each of which calls for slightly different methods of verification. Your friends are primarily interested in dichotomous forecasts. Quick research leads you to the fact that it is essentially binary prediction. 

Their terminology is slightly different. Following are the differences:
- Instead of True Positives (TP) --> they use _Hits_ (a), 
- Instead of False Positives (FP) --> they use _False Alarms_ (b)
- Instead of False Negatives (FN) --> they use _Misses_ (c), 
- Instead of True Negatives (TN) --> they use _Correct Rejects_ (d).

For every single model run, you are given:
1. a set of observations (Event [1] or No Event [0]) 
2. a set of prediction scores (a float between 0 and 1) and an event threshold, where the predicted outcome will be 

<center>$\hat{y}= 
\begin{cases}
    1 \text{ (Event occurred)},& \text{if } score \geq threshold\\
    0 \text{ (No event),}      & \text{if } score < threshold
\end{cases} $ <center>

Note that observations are ground truth and prediction scores and threshold will be used for determining the predicted model output.

In [65]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Here are some test data you can use for this homework
Y_test = [0, 0, 1, 1, 0, 1, 0, 0, 0, 1] # observations
P_scores = [0.1, 0.32, 0.48, 0.9, 0.6, 0.55, 0.42, 0.37, 0.61, 0.66] # prediction scores
threshold = 0.5 # prediction threshold

P_scores1 = P_scores.copy()

## Question 1 (20 points)
Given prediction scores, threshold, and observations create a function to determine the elements of a confusion matrix. For ease of use, you will output a `dict` (dictionary) object instead of a 2-dimensional numpy array. Note that _positive_ corresponds to the event occurrence.

In [66]:
from sklearn.metrics import confusion_matrix


def binary_conf_matrix(observation, p_scores, threshold):
    '''Finds the entries (TP, FP, FN, TN) in a binary confusion matrix for forecast problems
    
    Parameters
    ----------
    observation: list
        list of observations (1 is there is event, 0 is there is no event)
    p_scores: list
        list of prediction scores (scores vary between 0 and 1)
    threshold: float
        threshold that will be used for binary outcome from prediction scores
    Returns
    -------
    dict
        a dictionary that shows the counts for TP, TN, FP, and FNs.
    
    '''
    print('Empty dictionary with intial Values ')
    bcmd = {'TP':0, 'FN':0, 'FP':0, 'TN':0}
    print(bcmd)
    print('')
    
    for index, item in enumerate(P_scores):
        if item <0.5:
            P_scores[index] = 0
        elif item >= 0.5:
            P_scores[index] = 1
    print('')
    print('Updated P_scores based on threshold Values')
    print(P_scores)
    
    print('')
    print('Confusion Matrix is:')
    cm = confusion_matrix(P_scores,Y_test)
    print(cm)
    print('TP = ',cm[0][0])
    print('FP = ',cm[0][1])
    print('FN = ',cm[1][0])
    print('TN = ',cm[1][1])
    
    print('')
    bcmd = {'TP':cm[0][0],'FN':cm[0][1],'FP':cm[1][0],'TN':cm[1][1]}
    print('Updated Dictionary with Confusion Matrix values')
    print(bcmd)
    
    return bcmd

#test your code
bcmd = binary_conf_matrix( Y_test, P_scores, threshold )

Empty dictionary with intial Values 
{'TP': 0, 'FN': 0, 'FP': 0, 'TN': 0}


Updated P_scores based on threshold Values
[0, 0, 0, 1, 1, 1, 0, 0, 1, 1]

Confusion Matrix is:
[[4 1]
 [2 3]]
TP =  4
FP =  1
FN =  2
TN =  3

Updated Dictionary with Confusion Matrix values
{'TP': 4, 'FN': 1, 'FP': 2, 'TN': 3}


### Next six questions will use the same input parameters that are used in `binary_conf_matrix` function. Please refer to the Q1 starter code documentation for the input parameters of the Questions 2-7.

## Question 2 (20 points)

Create functions for calculating accuracy, precision, recall, and F1-score. You can use the definitions from slides (Chapter 10). (You are supposed to calculate the precision and recall (and thus F1-score) for 'Event' [1] class.)

In [67]:
total = sum(bcmd.values())
print('Sum of count in Confusion_Matrix is: ',total)
print('')


def accuracy(observation, p_scores, threshold):
    accuracy = (bcmd['TP'] + bcmd['TN'])/total
    return accuracy 

def precision(observation, p_scores, threshold):
    precision = bcmd['TP'] / (bcmd['TP']+bcmd['FP'])
    return precision

def recall(observation, p_scores, threshold):
    recall = bcmd['TP'] / (bcmd['TP']+bcmd['FN'])
    return recall

def f1score(observation, p_scores, threshold):
    f1score = float(2*precision*recall)/(precision+recall)
    return f1score


# test your code using the following
accuracy = accuracy( Y_test, P_scores, threshold )
print('Accucary is ',accuracy)
precision = precision( Y_test, P_scores, threshold )
print('Precision is ',precision)
recall = recall( Y_test, P_scores, threshold )
print('Recall is ',round(recall,2))
f1score = f1score( Y_test, P_scores, threshold )
print('F1_Score is ',round(f1score,2))

Sum of count in Confusion_Matrix is:  10

Accucary is  0.7
Precision is  0.6666666666666666
Recall is  0.8
F1_Score is  0.73


## Question 3 (10 points)
Calculate the bias score (BIAS). Bias score measures the ratio of the frequency of predicted event occurrences to the frequency of observed events. It can be calculated using the following formula:

$ BIAS = \frac{\text{hits} + \text{false alarms} }{ \text{hits} + \text{misses} }$

In [69]:
# bias score

bcmd1 = bcmd.copy()
print(bcmd1)

bcmd1['Hits'] = bcmd1.pop('TP')
bcmd1['Misses'] = bcmd1.pop('FN')
bcmd1['False_Alarm'] = bcmd1.pop('FP')
bcmd1['Correct_Rejects'] = bcmd1.pop('TN')
print(bcmd1)

def bias_score(observation, p_scores, threshold):
    bias = (bcmd1['Hits'] + bcmd1['False_Alarm']) / (bcmd1['Hits']+bcmd1['Misses'])
    return bias


# test your code using the following
bias = bias_score(Y_test, P_scores, threshold)
print('Bias is ',round(bias,2))

{'TP': 4, 'FN': 1, 'FP': 2, 'TN': 3}
{'Hits': 4, 'Misses': 1, 'False_Alarm': 2, 'Correct_Rejects': 3}
Bias is  1.2


## Question 4 (10 points)

Calculate the false alarm ratio (FAR). FAR is an indicator of what fraction of the predicted events actually did not occur (i.e., they were false alarms). You can calculate the FAR as follows: 

$ FAR = \frac{ \text{false alarms} }{ \text{hits} + \text{false alarms} }  $

In [70]:
def far(observation, p_scores, threshold):
    #your code goes here
    far = bcmd1['False_Alarm']/(bcmd1['Hits'] + bcmd1['False_Alarm'])
    return far

# test your code using the following
far = far(Y_test, P_scores, threshold)
print('false alarm ratio is ', round(far,2))

false alarm ratio is  0.33


## Question 5 (10 points)
Calculate the threat score. Threat score (which is also referred to as critical success index -- CSI) indicates how well did the predicted event outcomes correspond to the observed events. It measures the fraction of actual __and/or__ predicted events that were correctly predicted. It can be thought of as the accuracy when the correct negatives (TN) have been removed from consideration. It can be calculated as follows:

$CSI = \frac{ \text{hits} }{ \text{hits} + \text{misses} + \text{false alarms} } $



In [71]:
def csi(observation, p_scores, threshold):
    csi = bcmd1['Hits']/(bcmd1['Hits']+bcmd1['Misses']+bcmd1['False_Alarm'])
    return csi 


# test your code using the following
threat_score = csi(Y_test, P_scores, threshold)
print('threat score is ',round(threat_score,2))

threat score is  0.57


## Question 6 (15 points)
Skill scores are often used to understand the improvement of model performance over a given baseline (often a hypothetical, predetermined random forecast result). The first skill score you will implement is _true skill statistic (TSS)_ (which is also known as Hanssen-Kuipers Skill Score. It can be used to understand how well the prediction model separate the events (detected) from the no events. TSS can be calculated as follows:

$ TSS = \frac{\text{hits} }{\text{hits} + \text{misses}} - \frac{\text{false alarms} }{\text{false alarms} + \text{correct negatives}} $

In [72]:
def tss(observation, p_scores, threshold):
    tss = (bcmd1['Hits']/(bcmd1['Hits']+bcmd1['Misses'])) - (bcmd1['False_Alarm']/(bcmd1['False_Alarm']+bcmd1['Correct_Rejects'])) 
    return tss

# test your code using the following
true_skill_statistic = tss(Y_test, P_scores, threshold)
print('true skill statistic is',round(true_skill_statistic,2))

true skill statistic is 0.4


## Question 7 (15 points)
The next skill score you will calculate is Gilbert Skill Score (also known as Equitable Threat Score). This is an interesting indicator of performance because it measures the fraction of observed and/or predicted events that were correctly predicted, adjusted for hits (correctly predicted events) associated with random chance. For example, it is easier to correctly predict rain occurrence in a wet climate than in a dry climate. In other words, GSS can answer how well the predicted event occurrences correspond to the observed events (accounting for correct predictions appearing due to chance). The Gilbert Skill Score can be calculated as follows:

$GSS = \frac{ \text{hits} - \text{hits}_{random}}{ \text{hits} + \text{misses} + \text{false alarms} - \text{hits}_{random} } $

where the random hits can be calculated as:

$ \text{hits}_{random} = \frac{ (\text{hits}+\text{misses} )* (\text{hits}+\text{false alarms} )}{total} $.

_total_ is sum of all the elements in confusion matrix. 

Hint: Notice the similarity between GSS and threat score.

In [73]:
def gss(observation, p_scores, threshold):
    total = sum(bcmd1.values()) 
    hits_random = (( bcmd1['Hits'] +bcmd1['Misses'] )*( bcmd1['Hits']+bcmd1['False_Alarm']))/total
    gss = (bcmd1['Hits'] - hits_random)/( bcmd1['Hits'] + bcmd1['Misses'] + bcmd1['False_Alarm'] - hits_random)
    return gss


# test your code using the following
Gilbert_Skill_Score = gss(Y_test, P_scores, threshold)
print('Gilbert Skill Score is ', Gilbert_Skill_Score)

Gilbert Skill Score is  0.25


## Bonus Question (10 points)

Your last task is to determine an optimal threshold based on prediction scores, observations, and a given performance measure. Create a function called `pick_threshold` which will pick the best prediction score threshold (that will return the highest performance measure value based on the given performance metric). Hint: Python allows you to pass a (performance measure) function (such as `tss`, `csi`, or `f1score`) to `pick_threshold`. 

In [77]:
def pick_threshold(observation, P_scores, mfunc):
    '''Finds the prediction score threshold that returns the highest performance measure value 
    based on a given measure.
    
    Parameters
    ----------
    observation: list
        list of observations (1 is there is event, 0 is there is no event)
    p_scores: list
        list of prediction scores (scores vary between 0 and 1)
    mfunc: function
        one of the performance measurement functions that will take (observation, p_scores, threshold) as
        the list of arguments. (Hint: one can call mfunc(observation, p_scores, threshold) within the 
        function scope.)
    Returns
    -------
    float
        threshold value that gives the best performance based on mfunc
    
    '''
    #your code goes here
    score_list =[]
    limit1 = []
    intial_score = mfunc(observation, P_scores, threshold)
    print('Intial Score is: ',intial_score)
    limit = threshold
    while(limit<=1):
        limit = limit+0.05
        print('New Limit is  ',limit)
        limit1.append(limit)
        score1 = mfunc(observation, P_scores, limit)
        print('new_score is ',score1)
        score_list.append(score1)
    print(score_list)
    print(limit1)
#test your code using the following
pick_threshold(Y_test, P_scores, csi)
print('-----------------------------')
pick_threshold(Y_test, P_scores, tss)


Intial Score is:  0.5714285714285714
New Limit is   0.55
new_score is  0.5714285714285714
New Limit is   0.6000000000000001
new_score is  0.5714285714285714
New Limit is   0.6500000000000001
new_score is  0.5714285714285714
New Limit is   0.7000000000000002
new_score is  0.5714285714285714
New Limit is   0.7500000000000002
new_score is  0.5714285714285714
New Limit is   0.8000000000000003
new_score is  0.5714285714285714
New Limit is   0.8500000000000003
new_score is  0.5714285714285714
New Limit is   0.9000000000000004
new_score is  0.5714285714285714
New Limit is   0.9500000000000004
new_score is  0.5714285714285714
New Limit is   1.0000000000000004
new_score is  0.5714285714285714
[0.5714285714285714, 0.5714285714285714, 0.5714285714285714, 0.5714285714285714, 0.5714285714285714, 0.5714285714285714, 0.5714285714285714, 0.5714285714285714, 0.5714285714285714, 0.5714285714285714]
[0.55, 0.6000000000000001, 0.6500000000000001, 0.7000000000000002, 0.7500000000000002, 0.8000000000000003,